In [6]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import csv
import time

In [7]:
# Cấu hình Selenium WebDriver (với Chrome)
chrome_options = Options()
chrome_options.add_argument("--headless")  # chạy chế độ không giao diện để tăng hiệu suất
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Khởi tạo trình điều khiển Chrome
service = Service(r'C:\\Users\\HUUHUY\\Downloads\\chromedriver-win64\\chromedriver.exe')   # Thay bằng đường dẫn đến file chromedriver của bạn
# service = Service(r'C:\\Users\\HUUHUY\\Downloads\\App\\chrome-win64')   # Thay bằng đường dẫn đến file chromedriver của bạn
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL của trang chính
base_url = "https://e.vnexpress.net/"


In [8]:
def crawl_category(category_url, category_name, click_limit=30):
    articles = []
    
    # Mở trang danh mục bằng Selenium
    driver.get(category_url)
    time.sleep(2)  # Đợi trang tải

    # Tự động nhấn "view more stories" với giới hạn số lần
    click_count = 0
    while click_count < click_limit:
        try:
            # Nhấn nút "view more stories"
            view_more_button = driver.find_element(By.CLASS_NAME, "block_viewall")
            view_more_button.click()
            time.sleep(2)  # Nghỉ ngơi giữa các lần nhấn để tránh quá tải server
            click_count += 1  # Tăng biến đếm sau mỗi lần nhấn
        except:
            print("Không có nút 'view more stories' hoặc đã tải hết dữ liệu.")
            break  # Thoát khỏi vòng lặp nếu không có nút hoặc đã tải hết dữ liệu

    # Sau khi tải xong, lấy dữ liệu bằng BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles_list = soup.find_all('div', class_='item_news')
    
    for article in articles_list:
        try:
            # Kiểm tra thẻ tiêu đề, thử thẻ h4 trước, nếu không có thì chuyển sang thẻ h2
            title_tag = article.find('h4', class_='title_news_site') or article.find('h2', class_='title_news_site')
            title = title_tag.get_text(strip=True) if title_tag else None
            
            link = article.find('a')['href'] if article.find('a') else None
            img = article.find('img')['src'] if article.find('img') else None

            # Gửi yêu cầu tới trang chi tiết bài báo để lấy nội dung đầy đủ
            detail_response = requests.get(link)
            detail_soup = BeautifulSoup(detail_response.content, 'html.parser')

            # Lấy thông tin từ thẻ <p> và ghép lại
            info_paragraphs = detail_soup.find_all('span', class_='lead_post_detail')
            info = ' '.join([p.get_text(strip=True) for p in info_paragraphs])
            
            # Lấy nội dung chi tiết từ thẻ "fck_detail"
            detail_content = detail_soup.find('div', class_='fck_detail')
            info_detail = ' '.join([p.get_text(strip=True) for p in detail_content.find_all('p')]) if detail_content else None

            # Lưu thông tin bài báo
            articles.append({
                'title': title,
                'link': link,
                'img': img,
                'info': info,
                'info_detail': info_detail,
                'page': base_url,
                'category': category_name
            })
        except Exception as e:
            print(f"Error processing article: {e}")
            continue
    
    return articles


In [9]:

# Tạo file CSV và ghi dữ liệu
def save_to_csv(articles):
    keys = articles[0].keys()
    with open('vnexpress_articles.csv', 'w', newline='', encoding='utf-8-sig') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(articles)




In [10]:
# Danh sách danh mục và link của VnExpress
categories = {
    'economy': 'https://e.vnexpress.net/news/business/economy',
    'markets': 'https://e.vnexpress.net/news/business/markets',
    'property': 'https://e.vnexpress.net/news/business/property',
    'data-speaks': 'https://e.vnexpress.net/news/business/data-speaks',
    'Food-recipes': 'https://e.vnexpress.net/news/travel/food-recipes',
    'places': 'https://e.vnexpress.net/news/travel/places',
    'Brainteaser': 'https://e.vnexpress.net/news/news/brainteaser',
    'crime': 'https://e.vnexpress.net/news/news/crime',
    'traffic': 'https://e.vnexpress.net/news/news/traffic',
    'environment': 'https://e.vnexpress.net/news/news/environment',
    'Education': 'https://e.vnexpress.net/news/news/education',
}

# Gọi hàm crawl cho từng danh mục và lưu kết quả vào CSV
all_articles = []
for category_name, category_url in categories.items():
    print(f'Crawling category: {category_name}')
    articles = crawl_category(category_url, category_name)
    all_articles.extend(articles)
    time.sleep(1)  # Để tránh quá tải server


Crawling category: economy
Crawling category: markets
Crawling category: property
Crawling category: data-speaks
Crawling category: Food-recipes
Crawling category: places
Crawling category: Brainteaser
Crawling category: crime
Crawling category: traffic
Crawling category: environment
Crawling category: Education


In [11]:
# Lưu tất cả bài báo vào file CSV
if all_articles:
    save_to_csv(all_articles)
    print("Crawl thành công, dữ liệu đã được lưu vào 'vnexpress_articles.csv'")
else:
    print("Không tìm thấy bài báo nào.")

# Đóng trình điều khiển sau khi hoàn tất
driver.quit()

Crawl thành công, dữ liệu đã được lưu vào 'vnexpress_articles.csv'
